In [108]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


In [109]:
diabetes = pd.read_csv("data/diabetes_prediction_dataset.csv")

diabetes = diabetes[["gender", "Age", "BMI", "blood_glucose_levels", "diabetes"]]

diabetes

,gender,Age,BMI,blood_glucose_levels,diabetes
0,Female,67.0,27.32,80,0.0
1,Female,58.0,27.32,90,0.0
2,Male,2.0,18.41,100,0.0
3,Male,26.0,27.32,100,0.0
4,Female,61.0,24.07,160,0.0
...,...,...,...,...,...
49995,Female,80.0,27.32,90,0.0
49996,Female,2.0,17.37,100,0.0
49997,Male,66.0,27.83,155,0.0
49998,Female,24.0,35.42,100,0.0


In [45]:
diabetes.to_csv("data/diabetes_tidied.csv")

In [56]:
diabetes_tidied = pd.read_csv("diabetes_tidied.csv")

In [110]:
diabetes_train, diabetes_test = train_test_split(diabetes_tidied, train_size=.75)

In [111]:
diabetes_train.to_csv("data/diabetes_tidied_train.csv")

In [112]:
diabetes_train = diabetes_train[["gender", "Age", "BMI", "blood_glucose_levels", "diabetes"]]

In [113]:
diabetes_test = diabetes_test[["gender", "Age", "BMI", "blood_glucose_levels", "diabetes"]]

In [114]:
diabetes_train.to_csv("data/diabetes_tidied_train.csv")

In [115]:
diabetes_test.to_csv("data/diabetes_tidied_test.csv")

In [116]:
diabetes_train = diabetes_train.reset_index(drop=True)

In [117]:
diabetes_train

,gender,Age,BMI,blood_glucose_levels,diabetes
0,Female,3.0,15.78,158,0.0
1,Female,62.0,28.09,140,0.0
2,Male,59.0,37.51,130,1.0
3,Male,79.0,24.84,159,0.0
4,Female,2.0,27.32,200,0.0
...,...,...,...,...,...
37495,Female,22.0,27.32,200,0.0
37496,Male,54.0,27.32,160,0.0
37497,Male,58.0,27.02,200,0.0
37498,Female,25.0,39.71,159,0.0


**This marks the beginning of Visualization and table analysis**

In [129]:
grouped = diabetes_train.groupby(['gender', 'diabetes']).size().reset_index(name='Count')

In [130]:
grouped

,gender,diabetes,Count
0,Female,0.0,20242
1,Female,1.0,1643
2,Male,0.0,14131
3,Male,1.0,1476
4,Other,0.0,8


In [131]:
diabetes_train

,gender,Age,BMI,blood_glucose_levels,diabetes
0,Female,3.0,15.78,158,0.0
1,Female,62.0,28.09,140,0.0
2,Male,59.0,37.51,130,1.0
3,Male,79.0,24.84,159,0.0
4,Female,2.0,27.32,200,0.0
...,...,...,...,...,...
37495,Female,22.0,27.32,200,0.0
37496,Male,54.0,27.32,160,0.0
37497,Male,58.0,27.02,200,0.0
37498,Female,25.0,39.71,159,0.0


In [151]:
diabetes_female = diabetes_train[diabetes_train['gender'] == "Female"]
diabetes_male = diabetes_train[diabetes_train['gender'] == "Male"]

In [160]:
avg_bmi_FWdiabetes = diabetes_female[diabetes_female['diabetes'] == 1.0]["BMI"].mean()
avg_bmi_FWOUTdiabetes = diabetes_female[diabetes_female['diabetes'] == 0.0]["BMI"].mean()
avg_BGL_FWdiabetes= diabetes_female[diabetes_female['diabetes'] == 1.0]["blood_glucose_levels"].mean()
avg_BGL_FWOUTdiabetes= diabetes_female[diabetes_female['diabetes'] == 0.0]["blood_glucose_levels"].mean()

avg_bmi_MWdiabetes = diabetes_male[diabetes_male['diabetes'] == 1.0]["BMI"].mean()
avg_bmi_MWOUTdiabetes = diabetes_male[diabetes_male['diabetes'] == 0.0]["BMI"].mean()
avg_BGL_MWdiabetes = diabetes_male[diabetes_male['diabetes'] == 1.0]["blood_glucose_levels"].mean()
avg_BGL_MWOUTdiabetes = diabetes_male[diabetes_male['diabetes'] == 0.0]["blood_glucose_levels"].mean()

avg_bmi_MWdiabetes = diabetes_male[diabetes_male['diabetes'] == 1.0]["Age"].mean()
avg_bmi_MWOUTdiabetes = diabetes_male[diabetes_male['diabetes'] == 0.0]["Age"].mean()

avg_AGE_FWdiabetes= diabetes_female[diabetes_female['diabetes'] == 1.0]["Age"].mean()
avg_AGE_FWOUTdiabetes= diabetes_female[diabetes_female['diabetes'] == 0.0]["Age"].mean()

In [161]:
grouped = grouped.assign(BMI_averages = [avg_bmi_FWOUTdiabetes, avg_bmi_FWdiabetes, avg_bmi_MWOUTdiabetes, avg_bmi_MWdiabetes, 0])
grouped = grouped.assign(BGL_averages = [avg_BGL_FWOUTdiabetes, avg_BGL_FWdiabetes, avg_BGL_MWOUTdiabetes, avg_BGL_MWdiabetes, 0])
grouped = grouped.assign(AGE_averages = [avg_AGE_FWOUTdiabetes, avg_AGE_FWdiabetes, avg_bmi_MWOUTdiabetes, avg_bmi_MWdiabetes, 0])

grouped

,gender,diabetes,Count,BMI_averages,BGL_averages,AGE_averages
0,Female,0.0,20242,27.080237,133.110216,40.957008
1,Female,1.0,1643,32.254638,193.609860,61.216068
2,Male,0.0,14131,39.111132,133.326233,39.111132
3,Male,1.0,1476,60.652439,193.553523,60.652439
4,Other,0.0,8,0.000000,0.000000,0.000000


*The table below named GROUPED is the final table has summarized significant values within the dataset.*

In [162]:
grouped

,gender,diabetes,Count,BMI_averages,BGL_averages,AGE_averages
0,Female,0.0,20242,27.080237,133.110216,40.957008
1,Female,1.0,1643,32.254638,193.609860,61.216068
2,Male,0.0,14131,39.111132,133.326233,39.111132
3,Male,1.0,1476,60.652439,193.553523,60.652439
4,Other,0.0,8,0.000000,0.000000,0.000000
